# Etude de décarboxylases au sein de la souce PKT pour expliquer l'excès de CO2

In [20]:
import cobra
import pandas as pd
import escher
import cameo
%matplotlib inline
from math import fabs

In [21]:
def decarbox_reacts(model, result):
    df_decarbox = pd.DataFrame(columns=['flux_co2', 'reaction', 'name', 'genes'])
    df_loci = pd.read_excel('/Users/aalarcon/Documents/Jupyter/3_Isopropanol/files/data_model.xlsx', sheetname='genes', index_col=0)[['name1']]
    
    for r in model.metabolites.co2_c.reactions:
        if fabs(result.data_frame.loc[r.id, 'flux'])>1:
            flux = result.data_frame.loc[r.id, 'flux']*r.get_coefficient('co2_c')
            genes = ', '.join(g.id for g in r.genes)
            try:
                genes = ', '.join([df_loci.loc[g.id,'name1'] for g in r.genes])
            except:
                genes = ''
            df_decarbox.loc[r.id] = [float(format(flux,'.2f')), r.reaction, r.name, genes]
            
    return df_decarbox.sort_values('flux_co2', ascending=False)

In [22]:
df_loci = pd.read_excel('/Users/aalarcon/Documents/Jupyter/3_Isopropanol/files/data_model.xlsx', sheetname='genes', index_col=0)[['name1']]

In [23]:
wd = '/Users/aalarcon/Documents/Jupyter/3_Isopropanol/'

In [24]:
model = cobra.io.read_sbml_model(wd+'input/model_PKT_iprop.xml')

In [25]:
len(model.metabolites.co2_c.reactions)

81

--> Le modèle contient 81 réactions qui utilisent du co2 comme substrat ou produit.

In [26]:
cameo.phenotypic_phase_plane(model,'Biomass_WT', 'EX_co2').plot()

### Maximal Growth

In [27]:
res0 = cameo.pfba(model, objective='Biomass_WT')
df_aux = decarbox_reacts(model, res0)

In [34]:
df_aux

,flux_co2,reaction,name,genes
ICDHyr,111.89,icit_c + nadp_c <=> akg_c + co2_c + nadph_c,isocitrate dehydrogenase (NADP),icd
AKGDH,101.46,akg_c + coa_c + nad_c <=> co2_c + nadh_c + suc...,2-Oxogluterate dehydrogenase,"sucB, lpdA, sucA"
ACLS,8.14,h_c + 2.0 pyr_c --> alac_S_c + co2_c,acetolactate synthase,"ilvN, ilvI, ilvH, ilvB"
PDH,4.77,coa_c + nad_c + pyr_c --> accoa_c + co2_c + na...,pyruvate dehydrogenase,"aceE, lpdA, aceF"
OMCDC,4.20,3c4mop_c + h_c --> 4mop_c + co2_c,2-Oxo-4-methyl-3-carboxypentanoate decarboxyla...,
DAPDC,3.20,26dap_M_c + h_c --> co2_c + lys_L_c,diaminopimelate decarboxylase,lysA
OMPDC,3.07,h_c + orot5p_c --> co2_c + ump_c,orotidine-5'-phosphate decarboxylase,pyrF
ACHBS,2.71,2obut_c + h_c + pyr_c --> 2ahbut_c + co2_c,2-aceto-2-hydroxybutanoate synthase,"ilvN, ilvI, ilvH, ilvB"
PPNDH,1.73,h_c + pphn_c --> co2_c + h2o_c + phpyr_c,prephenate dehydratase,pheA
PPND,1.29,nad_c + pphn_c --> 34hpp_c + co2_c + nadh_c,prephenate dehydrogenase,tyrA


#### En croissance maximale, CO2 est produit en grande quantité par le cycle de Krebs.
#### Il est consommé par PPC et carbamate kinase

### Maximal CO2 production

In [9]:
res1 = cameo.pfba(model, objective='EX_co2')
print('CO2 export:', format(res1.data_frame.loc['EX_co2', 'flux'], '.2f'))

CO2 export: 600.00


In [10]:
df_res1 = decarbox_reacts(model, res1)

,flux_co2,reaction,name,genes
ICDHyr,250.00,icit_c + nadp_c <=> akg_c + co2_c + nadph_c,isocitrate dehydrogenase (NADP),icd
AKGDH,184.38,akg_c + coa_c + nad_c <=> co2_c + nadh_c + suc...,2-Oxogluterate dehydrogenase,"lpdA, sucA, sucB"
PDH,100.00,coa_c + nad_c + pyr_c --> accoa_c + co2_c + na...,pyruvate dehydrogenase,"lpdA, aceE, aceF"
GLUDC,65.62,glu_L_c + h_c --> 4abut_c + co2_c,Glutamate Decarboxylase,"gadA, gadB"
CO2tpp,-600.00,co2_p <=> co2_c,CO2 transporter via diffusion (periplasm),


### Glutamate Décarboxylase ??

Je prends max

#### Si je bloque ces 4 réactions ?

In [11]:
blocked_reacts = list(df_res1[df_res1['flux_co2']>0].index)
for r in blocked_reacts:
    model.reactions.get_by_id(r).bounds=(0,0)

In [12]:
res2 = cameo.pfba(model, objective='EX_co2')
print('CO2 export:', format(res2.data_frame.loc['EX_co2', 'flux'], '.2f'))

CO2 export: 600.00


In [13]:
decarbox_reacts(model, res2)

,flux_co2,reaction,name,genes
ME2,150.00,mal_L_c + nadp_c --> co2_c + nadph_c + pyr_c,malic enzyme (NADP),maeB
AOBUTDs,100.00,2aobut_c + h_c --> aact_c + co2_c,L-2-amino-3-oxobutanoate decarboxylation (spon...,
POX,67.48,h2o_c + pyr_c + q8_c --> ac_c + co2_c + q8h2_c,pyruvate oxidase,poxB
CO2tpp,-317.48,co2_p <=> co2_c,CO2 transporter via diffusion (periplasm),


## Visualisation des résultats escher

In [ ]:
res = res1
b=escher.Builder(map_json=wd+'files/Map_EM_iprop.json', reaction_data=res.data_frame.to_dict()['flux'],
                model=model)
b.display_in_browser()


Note: You must interrupt the kernel to end this command

Serving to http://127.0.0.1:7655/
[Ctrl-C to exit from terminal, or Ctrl-M i i to interrupt notebook kernel]


127.0.0.1 - - [16/Oct/2017 15:31:31] "GET / HTTP/1.1" 200 -
